# Задание 1. Загрузка данных
Изучить представленный набор данных на основе описания его столбцов в файле “horse_data.names” , загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.


In [1]:
import pandas as pd
import numpy as np

In [12]:
df = pd.read_csv("D:\\Нетология python\\Статистика_в_Python\\поиск_аномалий_в_данных_и_работа_с_пропущенными_значениями\\horse_data.csv", usecols=[1,2,4,5,6,7,11,23],names=['surgery?', 'Age', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome'],na_values=["",'',"NA",'?',"?"])


In [13]:
df.head()

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1,530101,66.0,28.0,3.0,3.0,4.0,2
1,1,534817,88.0,20.0,NaN,NaN,4.0,2
2,1,530334,40.0,24.0,1.0,1.0,3.0,2
3,9,5290409,164.0,84.0,4.0,1.0,4.0,1
4,1,530255,104.0,35.0,NaN,NaN,NaN,2


# Задание 2. Первичное изучение данных
Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

==========================================================================================================

In [32]:
# Создаем пустой словарь для статистик
statistics = []

# Разделим столбцы на категориальные и непрерывные
categorical_columns = ['surgery?', 'Age', 'temperature of extremities', 'pain', 'outcome']
continuous_columns = ['rectal temperature', 'pulse', 'respiratory rate']

# Для категориальных столбцов
for column in categorical_columns:
    mode = df[column].mode().values[0]  # Находим моду
    value_counts = df[column].value_counts()  # Анализ распределения по категориям
    unique_values = df[column].unique()  # Проверяем уникальные значения на предмет ошибок и соответствия описанию данных
    statistics.append({'Column': column, 'Type': 'Categorical', 'Mode': mode, 'Value Counts': value_counts, 'Unique Values': unique_values})

# Для непрерывных столбцов
for column in continuous_columns:
    describe = df[column].describe()  # Находим базовые статистики
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1  # Рассчитываем межквартильный размах
    outliers = df[column][(df[column] < q1 - 1.5*iqr) | (df[column] > q3 + 1.5*iqr)]  # Находим выбросы 
    statistics.append({'Column': column, 'Type': 'Continuous', 'Describe': describe, 'IQR': iqr, 'Outliers': outliers})

# Создаем DataFrame из словаря статистик
statistics_df = pd.DataFrame(statistics)


In [33]:
statistics_df

,Column,Type,Mode,Value Counts,Unique Values,Describe,IQR,Outliers
0,surgery?,Categorical,1.0,"surgery? 1 276 9 24 Name: count, dtype:...","[1, 9]",NaN,NaN,NaN
1,Age,Categorical,527544.0,Age 532349 2 528151 2 5279822 2 528...,"[530101, 534817, 530334, 5290409, 530255, 5283...",NaN,NaN,NaN
2,temperature of extremities,Categorical,1.0,temperature of extremities 1.0 184 3.0 1...,"[3.0, 1.0, 4.0, 2.0]",NaN,NaN,NaN
3,pain,Categorical,3.0,pain 3.0 172 4.0 73 1.0 39 2.0 ...,"[4.0, 3.0, 1.0, 2.0]",NaN,NaN,NaN
4,outcome,Categorical,1.0,"outcome 1 191 2 109 Name: count, dtype: ...","[2, 1]",NaN,NaN,NaN
5,rectal temperature,Continuous,NaN,NaN,NaN,count 300.000000 mean 71.280000 std ...,40.00,3 164.0 41 150.0 55 160.0 255 ...
6,pulse,Continuous,NaN,NaN,NaN,count 300.000000 mean 29.273333 std ...,14.25,3 84.0 20 60.0 23 60.0 39 96....
7,respiratory rate,Continuous,NaN,NaN,NaN,count 300.000000 mean 2.470000 std ...,2.00,"Series([], Name: respiratory rate, dtype: floa..."


# ==============================================================

# Задание 3. Работа с пропусками
Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

==========================================================================================================================


In [34]:
df = pd.read_csv("D:\\Нетология python\\Статистика_в_Python\\поиск_аномалий_в_данных_и_работа_с_пропущенными_значениями\\horse_data.csv",
                 na_values="?",  # Замена "?" на NaN
                 usecols=[1,2,4,5,6,7,11,23],
                 names=['surgery?', 'Age', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome'])

# Преобразование строковых значений в числа при необходимости
for column in df.columns:
    if df[column].dtype == 'object':
        # Преобразование строковых значений в числа, если возможно
        try:
            df[column] = pd.to_numeric(df[column])
        except ValueError:
            print(f"Невозможно преобразовать столбец '{column}' в числовой формат.")

# Вывод таблицы без знаков "?"
df


,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1,530101,66.0,28.0,3.0,3.0,4.0,2
1,1,534817,88.0,20.0,NaN,NaN,4.0,2
2,1,530334,40.0,24.0,1.0,1.0,3.0,2
3,9,5290409,164.0,84.0,4.0,1.0,4.0,1
4,1,530255,104.0,35.0,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...
295,1,533886,120.0,70.0,4.0,NaN,4.0,2
296,1,527702,72.0,24.0,3.0,2.0,3.0,1
297,1,529386,72.0,30.0,4.0,3.0,4.0,1
298,1,530612,100.0,24.0,3.0,3.0,3.0,1


In [41]:
df.isna().mean() * 100

surgery?                      0.0
Age                           0.0
rectal temperature            0.0
pulse                         0.0
respiratory rate              0.0
temperature of extremities    0.0
pain                          0.0
outcome                       0.0
dtype: float64

In [35]:
# Статистика до обработки
pre_processing_statistics = df.describe()

In [36]:
# Заполнение пропусков модой
categorical_columns = ['surgery?', 'Age', 'temperature of extremities', 'pain', 'outcome']
for column in categorical_columns:
    # Заменяем пропуски в столбце модой
    df[column].fillna(df[column].mode()[0], inplace=True)
df

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1,530101,66.0,28.0,3.0,3.0,4.0,2
1,1,534817,88.0,20.0,NaN,1.0,4.0,2
2,1,530334,40.0,24.0,1.0,1.0,3.0,2
3,9,5290409,164.0,84.0,4.0,1.0,4.0,1
4,1,530255,104.0,35.0,NaN,1.0,3.0,2
...,...,...,...,...,...,...,...,...
295,1,533886,120.0,70.0,4.0,1.0,4.0,2
296,1,527702,72.0,24.0,3.0,2.0,3.0,1
297,1,529386,72.0,30.0,4.0,3.0,4.0,1
298,1,530612,100.0,24.0,3.0,3.0,3.0,1


In [37]:
# Заполнение пропусков медианой
continuous_columns = ['rectal temperature', 'pulse', 'respiratory rate']
for column in continuous_columns:
    # Заменяем пропуски в столбце медианой
    df[column].fillna(df[column].median(), inplace=True)
df

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1,530101,66.0,28.0,3.0,3.0,4.0,2
1,1,534817,88.0,20.0,3.0,1.0,4.0,2
2,1,530334,40.0,24.0,1.0,1.0,3.0,2
3,9,5290409,164.0,84.0,4.0,1.0,4.0,1
4,1,530255,104.0,35.0,3.0,1.0,3.0,2
...,...,...,...,...,...,...,...,...
295,1,533886,120.0,70.0,4.0,1.0,4.0,2
296,1,527702,72.0,24.0,3.0,2.0,3.0,1
297,1,529386,72.0,30.0,4.0,3.0,4.0,1
298,1,530612,100.0,24.0,3.0,3.0,3.0,1


In [38]:
# # Статистика до обработки
# pre_processing_statistics = df.describe()

# Заполнение пропусков

# Статистика после обработки
post_processing_statistics = df.describe()

# Сравнение статистик

In [39]:
print("Статистика до обработки:")
pre_processing_statistics

Статистика до обработки:


,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
count,300.000000,3.000000e+02,276.000000,242.000000,244.000000,231.000000,256.000000,300.000000
mean,1.640000,1.085889e+06,71.913043,30.417355,2.348361,2.017316,2.917969,1.363333
std,2.173972,1.529801e+06,28.630557,17.642231,1.045054,1.042428,0.976744,0.481763
min,1.000000,5.184760e+05,30.000000,8.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,5.289040e+05,48.000000,18.500000,1.000000,1.000000,3.000000,1.000000
50%,1.000000,5.303055e+05,64.000000,24.500000,3.000000,2.000000,3.000000,1.000000
75%,1.000000,5.347275e+05,88.000000,36.000000,3.000000,3.000000,4.000000,2.000000
max,9.000000,5.305629e+06,184.000000,96.000000,4.000000,4.000000,4.000000,2.000000


In [40]:
print("\nСтатистика после обработки:")
post_processing_statistics


Статистика после обработки:


,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
count,300.000000,3.000000e+02,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,1.640000,1.085889e+06,71.280000,29.273333,2.470000,1.783333,2.930000,1.363333
std,2.173972,1.529801e+06,27.541545,16.010979,0.975845,1.009846,0.902486,0.481763
min,1.000000,5.184760e+05,30.000000,8.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,5.289040e+05,48.000000,20.000000,1.000000,1.000000,3.000000,1.000000
50%,1.000000,5.303055e+05,64.000000,24.500000,3.000000,1.000000,3.000000,1.000000
75%,1.000000,5.347275e+05,88.000000,34.250000,3.000000,3.000000,3.000000,2.000000
max,9.000000,5.305629e+06,184.000000,96.000000,4.000000,4.000000,4.000000,2.000000


describe() выводит стандартные статистические характеристики, такие как среднее значение, стандартное отклонение, минимум, максимум, медиану и квартили. Если замена пропущенных значений происходит с использованием стандартных методов (например, медианы или среднего), статистические показатели по сути не будут меняться.